Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [23]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-01-25 01:02:26--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.38.166
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.38.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  73.4MB/s    in 2.7s    

2020-01-25 01:02:28 (73.4 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [24]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [25]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01/instacart_2017_05_01


In [26]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
##### YOUR CODE HERE #####
import pandas as pd
import numpy as np

##Checking the datasets

In [28]:
aisles=pd.read_csv('aisles.csv')
aisles.sample(5)

,aisle_id,aisle
44,45,candy chocolate
127,128,tortillas flat bread
50,51,preserved dips spreads
23,24,fresh fruits
7,8,bakery desserts


In [29]:
departments=pd.read_csv('departments.csv')
departments.sample(5)

,department_id,department
12,13,pantry
5,6,international
19,20,deli
20,21,missing
7,8,pets


In [30]:
order_products__prior=pd.read_csv('order_products__prior.csv')
order_products__prior.sample(5)

,order_id,product_id,add_to_cart_order,reordered
29886163,3152200,20947,5,1
19683639,2076006,18978,10,1
2636551,278070,5450,9,0
32152527,3391446,41665,7,1
30198586,3185446,11865,37,0


In [31]:
order_products__train=pd.read_csv('order_products__train.csv')
order_products__train.sample(5)

,order_id,product_id,add_to_cart_order,reordered
418212,1024382,30827,21,0
554136,1362942,17322,11,1
738439,1816853,14536,5,1
1353565,3344284,11140,6,1
279794,681899,5157,4,1


In [32]:
orders=pd.read_csv('orders.csv')
orders.sample(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3125118,1985526,188408,prior,2,0,14,15.0
3121463,3307460,188184,prior,6,2,21,13.0
2514876,2648561,151392,prior,7,4,15,4.0
1863542,1186087,111842,prior,11,2,19,8.0
1425499,1949530,85646,prior,36,4,15,3.0


In [33]:
products=pd.read_csv('products.csv')
products.sample(5)

,product_id,product_name,aisle_id,department_id
37053,37054,5 Cheese Garlic Bread Loaf,129,1
35508,35509,Mesquite with Lime Juice Marinade,5,13
34037,34038,Fresh Grated Prepared Horseradish & Beets,72,13
5598,5599,Roll On Deodorant Crystal,80,11
39493,39494,Delicate Blossom Cleansing Wash,126,11


In [34]:
# Let's find the "Bananas" in the list of products
# and figure out the product ID
products[products['product_name']=='Bananas']


,product_id,product_name,aisle_id,department_id
39275,39276,Bananas,24,4


##DataFrame

In [0]:
# Let's create a dataframe by concatenating
# the important data from other datasets.

#For that we will need the following columns:
# 'product_id' in "products"
# 'product_name' in "products"



In [35]:
# Let's check out datasets for more information
print('order_products__prior:',order_products__prior.shape)
print('order_products__train:',order_products__train.shape)

order_products__prior: (32434489, 4)
order_products__train: (1384617, 4)


In [36]:
# Create a new dataframe started by concatenating
# 'order_products__prior' and 'order_products__train'
df=pd.concat([order_products__prior,order_products__train], axis=0)
# I didn't need to type 'axis=0' since it's considered to be there by default
print(df.shape)
df.head(10)

(33819106, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [44]:
# How many times have bananas been purchased?
condition_ban=(df['product_id']==39276)
len(df[condition_ban])

5500

In [0]:
# Yay! It finally worked!
# Now let's make some conclusions.
# Apparently I'll need the following columns to finish the assignment:
# 1. [products['product_id'] and [products['product_name']
#     to associate product_id with a particular product
# 2. [df['add_to_cart_order'] and [df['reordered']
#     from the original dataframe

# Let's add the info from 'products' to the main dataframe

In [0]:
# Now let's start narrowing the data in other datasets
# before we start merging them together
condition_1=((products['product_name']=="Bananas")&(orders['order_number'])<=2)
orders_subset=orders[condition_1]
print(orders_subset.shape)
orders_subset.head()

In [0]:
# Checking how often particular products were ordered
con_banana=(aisles['aisle']==['banana'])
aisles[con_banana].head(10)

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

In [0]:
# table2 -> tidy

In [0]:
# tidy -> table2

In [0]:
flights = sns.load_dataset('flights')

In [0]:
# Flights Pivot Table

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####